In [2]:
!pip install unidecode

In [3]:
import pandas as pd
import os
import numpy as np
from itertools import *
import json
import unidecode
import unicodedata
from datetime import datetime

### DEFININDO ALGUMAS FUNÇÕES A SEREM UTILIZADAS PELO PROGRAMA

In [7]:
def path(name):
    path = os.path.join('out',name)
    return(path)


def read_csv(name):
    df = pd.read_csv(path(name), sep=',', encoding='ISO-8859-1')
    df['Data'] =pd.to_datetime(df['Data'])
    df.index = df['Data']
    #df = df.drop('Data' , axis = 1)
    df = df.drop('Data.1' , axis = 1)
    return df

def compair (coluna_a, coluna_b, dataframe_b):
    listas = coluna_a.tolist()

    for lista in listas:
        df = dataframe_b.loc[coluna_b == lista]
        print(dataframe_b.loc[coluna_b == lista])                      
    return df

def limpa_str(df):
    if type(df) == type(pd.DataFrame()):
        lista = df.columns.tolist()
        for i in lista:
            df[i] = df[i].str.rstrip()
    else:
        df = df.str.rstrip()

    return df

def contain(coluna_a,coluna_b,data_b):
    
    #coluna_a = limpa_str(coluna_a)
    #coluna_b = limpa_str(coluna_b)
    
    lista = coluna_a.tolist()
    df = data_b[coluna_b.apply(lambda x: x in lista)]
    return df


def contain2(coluna_a,coluna_b,data_b):
    
    #coluna_a = limpa_str(coluna_a)
    #coluna_b = limpa_str(coluna_b)
    
    lista = coluna_a.tolist()
    lista.remove(0)
    df = data_b[coluna_b.apply(lambda x: x in lista)]
    return df

def not_contain(coluna_b,coluna_a,data_b):
    
    #coluna_a = limpa_str(coluna_a)
    #coluna_b = limpa_str(coluna_b)
    
    lista = coluna_a.tolist()
    df = data_b[coluna_b.apply(lambda x: x not in lista)]
    return df



def to_dict(inputDF, modo_de_falha, dicionario, origin):
    for i in range(len(inputDF)):
        dicionario[inputDF.LocInstal[i]][inputDF.Data[i]] = {'Descricao': inputDF['Descrição'][i],  'Modo de Falha': modo_de_falha, 'Nota': inputDF['Nota'][i], 'Fonte': origin}
    return dicionario

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    only_ascii = only_ascii.decode("utf-8") 
    return only_ascii

### VERIFICANDO SE HÁ VALORES DUPLICADOS NAS ORDENS E NOTAS DE MANUTENÇÃO

In [8]:
name = 'dados_sap_out.csv'
data_sap = read_csv(name)

data_sap['Dup_Nota'] = list(data_sap.Nota.duplicated(keep=False))
data_sap['Dup_Ordem'] = list(data_sap.Ordem.duplicated(keep=False))

value1 = len(data_sap['Dup_Nota']) - data_sap['Dup_Nota'].value_counts()[0]
value2 = len(data_sap['Dup_Ordem']) - data_sap['Dup_Ordem'].value_counts()[0]

data_sap.fillna(value=0, inplace=True)
data_sap = data_sap.astype({"Nota": np.int64, "Ordem": np.int64})



print(f'Quantidade de valores duplicados em notas de manutenção:    {value1}')
print(f'Quantidade de valores duplicados em ordens de manutenção:    {value2}')

Quantidade de valores duplicados em notas de manutenção:    0
Quantidade de valores duplicados em ordens de manutenção:    258


In [9]:
name = 'tabela_de_intervencoes_out.csv'
data_inter = read_csv(name)


data_inter = data_inter.rename(columns={"Nota PM": "Nota"})

data_inter['Dup_Nota'] = list(data_inter.Nota.duplicated(keep=False))
data_inter['Dup_Ordem'] = list(data_inter.Ordem.duplicated(keep=False))

value1 = len(data_inter['Dup_Nota']) - data_inter['Dup_Nota'].value_counts()[0]
value2 = len(data_inter['Dup_Ordem']) - data_inter['Dup_Ordem'].value_counts()[0]

data_inter.fillna(value=0, inplace=True)
data_inter = data_inter.astype({"Nota": np.int64, "Ordem": np.int64})

data_inter.rename(columns={'Motivo':'Descrição', 'TAG': 'LocInstal'}, inplace=True)

print(f'Quantidade de valores duplicados em notas de manutenção:    {value1}')
print(f'Quantidade de valores duplicados em ordens de manutenção:    {value2}')

Quantidade de valores duplicados em notas de manutenção:    2
Quantidade de valores duplicados em ordens de manutenção:    0


### VERIFICANDO SE HÁ RELAÇÃO ENTRE OS VALORES DE ORDEM, NOTA E DATA ENTRE AS PLANILHAS DE INTERVENÇÕES E NOTAS SAP

In [10]:
contain2(data_inter['Ordem'],data_sap['Ordem'],data_sap)

,Data,LocInstal,Nota,Tp,DescriÃ§Ã£o,Notificador,Ordem,InÃ­c desej,Encerram,Concldesj,Status sistema,tempo_execucao,Dup_Nota,Dup_Ordem
Data,,,,,,,,,,,,,,
2011-02-23,2011-02-23,RS01-SC-302A,91000286226,Z1,C 302A revisar compressor,AUAD5,9346741,23.02.2011,2011-07-21,23.02.2013,MSEN ORDA,148 days 00:00:00.000000000,False,False
2013-04-13,2013-04-13,RS01-SC-302A,93000449954,Z3,REVISÃO GERAL TEMPERATURA ALTA DESCARGA,GIOVANI,50034193,01.06.2013,2014-07-22,31.12.2014,MSEN ORDA,465 days 00:00:00.000000000,False,False
2013-06-17,2013-06-17,RS01-SC-302A,93000467359,Z3,RETRABALHO - VIBRAÃÃO E RUÃDO ANORMAL,GIOVANI,50035144,17.06.2013,2013-06-26,17.07.2013,MSEN ORDA TMEE,9 days 00:00:00.000000000,False,False
2014-12-13,2014-12-13,RS01-SC-302A,98001525227,Z8,SC-302A - ManutenÃ§Ã£o Preventiva MecÃ¢nica,0,91368082,13.12.2014,2015-01-07,02.01.2015,MSEN ORDA,25 days 00:00:00.000000000,False,False
2015-01-07,2015-01-07,RS01-SC-302A,93000625408,Z3,PAROU POR VIBRAÃÃO ALTA,EDEROL01,9728834,07.01.2015,2015-03-11,07.02.2015,MSEN ORDA,63 days 00:00:00.000000000,False,False
2015-09-22,2015-09-22,RS01-SC-302A,93000695058,Z3,C302A-PressÃ£o alta descarga 1ÂºestÃ¡gio.,DENIR5,9812384,22.09.2015,2015-09-24,22.01.2016,MSEN ORDA TMEE,2 days 00:00:00.000000000,False,False
2016-04-05,2016-04-05,RS01-SC-302A,93000746899,Z3,Temperatura alta descarga - folga camisa,BRUNO,9870080,05.04.2016,2016-06-07,05.06.2016,MSEN ORDA,63 days 00:00:00.000000000,False,False
2016-05-01,2016-05-01,RS01-SC-302A,93000754065,Z3,C302A -> Alarmando vibraÃ§Ã£o no 1Âº estÃ¡gi,HANZ5,9880110,01.05.2016,2016-05-04,08.05.2016,MSEN ORDA TMEE,3 days 00:00:00.000000000,False,False
2018-02-26,2018-02-26,RS01-SC-302A,98002610243,Z8,SC-302A - ManutenÃ§Ã£o Preventiva MecÃ¢nica,0,92423478,26.02.2018,2018-06-15,01.01.2019,MSEN ORDA,109 days 00:00:00.000000000,False,False


In [11]:
contain2(data_inter['Nota'],data_sap['Nota'],data_sap)

,Data,LocInstal,Nota,Tp,DescriÃ§Ã£o,Notificador,Ordem,InÃ­c desej,Encerram,Concldesj,Status sistema,tempo_execucao,Dup_Nota,Dup_Ordem
Data,,,,,,,,,,,,,,
2013-04-13,2013-04-13,RS01-SC-302A,93000449954,Z3,REVISÃO GERAL TEMPERATURA ALTA DESCARGA,GIOVANI,50034193,01.06.2013,2014-07-22,31.12.2014,MSEN ORDA,465 days 00:00:00.000000000,False,False
2013-06-17,2013-06-17,RS01-SC-302A,93000467359,Z3,RETRABALHO - VIBRAÃÃO E RUÃDO ANORMAL,GIOVANI,50035144,17.06.2013,2013-06-26,17.07.2013,MSEN ORDA TMEE,9 days 00:00:00.000000000,False,False
2014-12-13,2014-12-13,RS01-SC-302A,98001525227,Z8,SC-302A - ManutenÃ§Ã£o Preventiva MecÃ¢nica,0,91368082,13.12.2014,2015-01-07,02.01.2015,MSEN ORDA,25 days 00:00:00.000000000,False,False
2015-01-07,2015-01-07,RS01-SC-302A,93000625408,Z3,PAROU POR VIBRAÃÃO ALTA,EDEROL01,9728834,07.01.2015,2015-03-11,07.02.2015,MSEN ORDA,63 days 00:00:00.000000000,False,False
2015-09-22,2015-09-22,RS01-SC-302A,93000695058,Z3,C302A-PressÃ£o alta descarga 1ÂºestÃ¡gio.,DENIR5,9812384,22.09.2015,2015-09-24,22.01.2016,MSEN ORDA TMEE,2 days 00:00:00.000000000,False,False
2016-04-05,2016-04-05,RS01-SC-302A,93000746899,Z3,Temperatura alta descarga - folga camisa,BRUNO,9870080,05.04.2016,2016-06-07,05.06.2016,MSEN ORDA,63 days 00:00:00.000000000,False,False
2016-05-01,2016-05-01,RS01-SC-302A,93000754065,Z3,C302A -> Alarmando vibraÃ§Ã£o no 1Âº estÃ¡gi,HANZ5,9880110,01.05.2016,2016-05-04,08.05.2016,MSEN ORDA TMEE,3 days 00:00:00.000000000,False,False
2018-02-26,2018-02-26,RS01-SC-302A,98002610243,Z8,SC-302A - ManutenÃ§Ã£o Preventiva MecÃ¢nica,0,92423478,26.02.2018,2018-06-15,01.01.2019,MSEN ORDA,109 days 00:00:00.000000000,False,False
2018-04-24,2018-04-24,RS01-SC-302A,91000615348,Z1,Temperatura alta descarga 2nd stg,BRUNO,92501127,24.04.2018,2018-06-21,24.08.2018,MSEN ORDA,58 days 00:00:00.000000000,False,False


In [12]:
contain(data_inter['Data'],data_sap['Data'],data_sap)

,Data,LocInstal,Nota,Tp,DescriÃ§Ã£o,Notificador,Ordem,InÃ­c desej,Encerram,Concldesj,Status sistema,tempo_execucao,Dup_Nota,Dup_Ordem
Data,,,,,,,,,,,,,,
2012-02-16,2012-02-16,RS01-SC-302A,93000350826,Z3,M- SC302A QUEBRA DO VIRABREQUIM,MENSCH3,50030426,16.02.2012,2012-03-14,17.02.2012,MSEN ORDA TMEE,27 days 00:00:00.000000000,False,False
2012-02-16,2012-02-16,RS01-SC-302A,93000350771,Z3,M- SC302A QUEBRA DO VIRABREQUINHO,MENSCH3,9426998,16.02.2012,2012-02-16,16.02.2013,MSEN ORDA,0 days 00:00:00.000000000,False,False
2013-06-17,2013-06-17,RS01-SC-302A,93000467359,Z3,RETRABALHO - VIBRAÃÃO E RUÃDO ANORMAL,GIOVANI,50035144,17.06.2013,2013-06-26,17.07.2013,MSEN ORDA TMEE,9 days 00:00:00.000000000,False,False
2016-01-31,2016-01-31,RS01-SC-302A,98001853973,Z8,PREV.LUB.VISUAL.NÃVEL SC-302A 2XSEMAN.,0,91645507,31.01.2016,2016-02-11,06.02.2016,MSEN ORDA,11 days 00:00:00.000000000,False,False
2018-04-24,2018-04-24,RS01-SC-302A,91000615348,Z1,Temperatura alta descarga 2nd stg,BRUNO,92501127,24.04.2018,2018-06-21,24.08.2018,MSEN ORDA,58 days 00:00:00.000000000,False,False
2018-05-10,2018-05-10,RS01-SC-302A,91000617583,Z1,C302A Vazamento HC flange descarga 2Â°est,BERVIG5,92523696,10.05.2018,2018-07-18,10.11.2018,MSEN ORDA,69 days 00:00:00.000000000,False,False
2018-11-19,2018-11-19,RS01-SC-302A,98002860481,Z8,SC-302A - ManutenÃ§Ã£o Preventiva MecÃ¢nica,AGUIAR3,92772081,19.11.2018,2019-01-23,19.11.2018,MSEN MSIM ORDA,65 days 00:00:00.000000000,False,False
2019-02-26,2019-02-26,RS01-SC-302A,98002959047,Z8,M-VIB-Medir VibraÃ§Ã£o RS01-SC-302A-15D,IP1020100803,0,13.03.2019,2019-03-13,13.03.2019,MSEN,15 days 00:00:00.000000000,False,True
2019-02-26,2019-02-26,RS01-SC-302A,91000710592,Z1,PSHH-3405 - parando do compressor,CAMBOIM5,92904423,26.02.2019,2019-02-26,26.11.2019,MSEN ORDA TMEE,0 days 00:00:00.000000000,False,False


### MODIFICAR PALAVRAS COM ACENTO

In [13]:
data_sap['Descrição'] = data_sap['Descrição'].str.lower()

data_sap['Descrição'] = data_sap['Descrição'].apply(remove_accents)

KeyError: 'Descrição'

In [6]:
data_inter['Descrição'] = data_inter['Descrição'].str.lower()

data_inter['Descrição'] = data_inter['Descrição'].apply(remove_accents)

### CONTAR AS PALAVRAS MAIS COMUNS

In [ ]:
'''
                NOTAS DE MANUTENÇÃO
'''
intermedio = pd.DataFrame(data=pd.Series(' '.join(data_sap['Descrição']).lower().split()).value_counts()[:200], columns=['ocorrencias'])
intermedio.reset_index(inplace=True)
intermedio['caracteres'] = intermedio['index'].str.len()
intermedio['index']
intermedio = intermedio[intermedio.caracteres > 2]
intermedio = intermedio[intermedio.ocorrencias > 1]


intermedio.reset_index(inplace=True, drop=True)

intermedio.to_csv('sap_common.csv', sep=';', encoding='ISO-8859-1')

In [ ]:
'''
                PLANILHA DE INTERVENÇÕES
'''
intermedio1 = pd.DataFrame(data=pd.Series(' '.join(data_inter['Descrição']).lower().split()).value_counts()[:200], columns=['ocorrencias'])
intermedio1.reset_index(inplace=True)
intermedio1['caracteres'] = intermedio1['index'].str.len()
intermedio1 = intermedio1[intermedio1.caracteres > 3]
intermedio1 = intermedio1[intermedio1.ocorrencias > 1]
intermedio1.reset_index(inplace=True, drop=True)

intermedio1.to_csv('interven_common.csv', sep=';', encoding='ISO-8859-1')

### GERANDO O JSON 

In [7]:
compressores = {
        "RS01-SC-301A": {

        },
        "RS01-SC-301B":{ 

        },
        'RS01-SC-302A':{ 

        },
        'RS01-SC-302B': {

        }
}

### CRIANDO FILTROS PARA ENCONTRAR PALAVRAS CHAVE PARA OS MODOS DE FALHAS
#### .1 Falha no conjunto de válvulas

In [8]:
'''
            VAZAMENTO DE GAS É FALHA DE VALVULA OU FOLGA ENTRE A CAMISA E O PISTÃO
'''

modo_de_falha = 'Falha no conjunto de valvulas'

cond_1 = data_sap['Descrição'].str.contains('vaz', case = False)
cond_2 = data_sap['Descrição'].str.contains('gas', case = False)
cond_3 = data_sap['Descrição'].str.contains('vaz-hc', case = False)
cond_4 = data_sap['Descrição'].str.contains('val', case = False)
cond_5 = data_sap['Descrição'].str.contains('temp', case = False)
cond_6 = data_sap['Descrição'].str.contains('folga', case = False)


condicional = (cond_1 | cond_2 | cond_3 | cond_4 | cond_5) & (~cond_6)

dataFRAME = data_sap[condicional]

print(len(dataFRAME))


dataFRAME['Data']=dataFRAME['Data'].astype(str)
dataFRAME['Nota']=dataFRAME['Nota'].astype(str)

origem = 'Notas de Manutenção SAP'
compressores = to_dict(dataFRAME, modo_de_falha, compressores, origem)

32


c:\users\eduardo.lima.k2\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\eduardo.lima.k2\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [9]:
'''
            VAZAMENTO DE GAS É FALHA DE VALVULA OU FOLGA ENTRE A CAMISA E O PISTÃO
'''

modo_de_falha = 'Falha no conjunto de valvulas'

cond_1 = data_inter['Descrição'].str.contains('vaz', case = False)
cond_2 = data_inter['Descrição'].str.contains('gas', case = False)
cond_3 = data_inter['Descrição'].str.contains('vaz-hc', case = False)
cond_4 = data_inter['Descrição'].str.contains('val', case = False)
cond_5 = data_inter['Descrição'].str.contains('temp', case = False)
cond_6 = data_inter['Descrição'].str.contains('folga', case = False)


condicional = (cond_1 | cond_2 | cond_3 | cond_4 | cond_5) & (~cond_6)

dataFRAME = data_inter[condicional]

print(len(dataFRAME))


dataFRAME['Data']=dataFRAME['Data'].astype(str)
dataFRAME['Nota']=dataFRAME['Nota'].astype(str)

origem = 'Tabela de intervenções'
compressores = to_dict(dataFRAME, modo_de_falha, compressores, origem)

5


c:\users\eduardo.lima.k2\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\eduardo.lima.k2\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


#### .2 Folga entre camisa e pistão

In [10]:
modo_de_falha = 'Folga entre camisa e pistão'

cond_1 = data_sap['Descrição'].str.contains('folga', case = False)

dataFRAME=data_sap[cond_1]

dataFRAME['Data']=dataFRAME['Data'].astype(str)
dataFRAME['Nota']=dataFRAME['Nota'].astype(str)
origem = 'Notas de Manutenção SAP'
compressores = to_dict(dataFRAME, modo_de_falha, compressores, origem)

c:\users\eduardo.lima.k2\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
c:\users\eduardo.lima.k2\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
modo_de_falha = 'Folga entre camisa e pistão'

cond_1 = data_inter['Descrição'].str.contains('folga', case = False)

dataFRAME=data_inter[cond_1]

dataFRAME['Data']=dataFRAME['Data'].astype(str)
dataFRAME['Nota']=dataFRAME['Nota'].astype(str)
origem = 'Tabela de intervenções'
compressores = to_dict(dataFRAME, modo_de_falha, compressores, origem)

#### .3 Eficiencia Volumétrica e Politrópica

#### .4 Eficiência energética do motor

In [12]:
'''
            VAZAMENTO DE ÓLEO REDUZ A EFICIENCIA DO MOTOR
'''
modo_de_falha = 'Eficiencia energetica do motor'

cond_1 = data_sap['Descrição'].str.contains('baixo', case = False)
cond_2 = data_sap['Descrição'].str.contains('oleo', case = False)
# df=data_sap[cond_1]
dataFRAME=data_sap[cond_1 | cond_2]

# dataFRAME

dataFRAME['Data']=dataFRAME['Data'].astype(str)
dataFRAME['Nota']=dataFRAME['Nota'].astype(str)
origem = 'Notas de Manutenção SAP'
compressores = to_dict(dataFRAME, modo_de_falha, compressores, origem)

c:\users\eduardo.lima.k2\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
c:\users\eduardo.lima.k2\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [13]:
'''
            VAZAMENTO DE ÓLEO REDUZ A EFICIENCIA DO MOTOR
'''
modo_de_falha = 'Eficiencia energetica do motor'

cond_1 = data_inter['Descrição'].str.contains('baixo', case = False)
cond_2 = data_inter['Descrição'].str.contains('oleo', case = False)
# df=data_sap[cond_1]
dataFRAME=data_inter[cond_1 | cond_2]

# dataFRAME

dataFRAME['Data']=dataFRAME['Data'].astype(str)
dataFRAME['Nota']=dataFRAME['Nota'].astype(str)
origem = 'Tabela de intervenções'
compressores = to_dict(dataFRAME, modo_de_falha, compressores, origem)

#### .5 Modelo de vribração

In [14]:
'''
        COLOCAR AS INFORMAÇÕES NO JSON
'''
modo_de_falha = 'Modelo de Vibracao'

# condicionais de vibração
cond_1 = data_sap['Descrição'].str.contains('vib', case = False)
cond_2 = data_sap['Descrição'].str.contains('m-vib', case = False)
cond_3 = data_sap['Descrição'].str.contains('sensor', case = False)
cond_4 = data_sap['Descrição'].str.contains('indevid', case = False)
cond_5 = data_sap['Descrição'].str.contains('paraf', case = False)
cond_6 = data_sap['Descrição'].str.contains('casa', case = False)


condicional = (cond_1 & ~cond_2 & ~cond_3 & ~cond_4) | (cond_5 & ~cond_6)

dataFRAME = data_sap[condicional]

dataFRAME['Data']=dataFRAME['Data'].astype(str)
dataFRAME['Nota']=dataFRAME['Nota'].astype(str)

origem = 'Notas de Manutenção SAP'
compressores = to_dict(dataFRAME, modo_de_falha, compressores, origem)

c:\users\eduardo.lima.k2\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\eduardo.lima.k2\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [15]:
'''
        COLOCAR AS INFORMAÇÕES NO JSON
'''
modo_de_falha = 'Modelo de Vibracao'

# condicionais de vibração
cond_1 = data_inter['Descrição'].str.contains('vib', case = False)
cond_2 = data_inter['Descrição'].str.contains('m-vib', case = False)
cond_3 = data_inter['Descrição'].str.contains('sensor', case = False)
cond_4 = data_inter['Descrição'].str.contains('indevid', case = False)
cond_5 = data_inter['Descrição'].str.contains('paraf', case = False)
cond_6 = data_inter['Descrição'].str.contains('casa', case = False)


condicional = (cond_1 & ~cond_2 & ~cond_3 & ~cond_4) | (cond_5 & ~cond_6)

dataFRAME = data_inter[condicional]

dataFRAME['Data']=dataFRAME['Data'].astype(str)
dataFRAME['Nota']=dataFRAME['Nota'].astype(str)
origem = 'Tabela de intervenções'
compressores = to_dict(dataFRAME, modo_de_falha, compressores, origem)

c:\users\eduardo.lima.k2\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\eduardo.lima.k2\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### GERAR O JSON

In [16]:
print(json.dumps(compressores, indent=4))

{
    "RS01-SC-301A": {},
    "RS01-SC-301B": {},
    "RS01-SC-302A": {
        "2009-07-06": {
            "Descricao": "sanar vazamento de oleo-substituir selo",
            "Modo de Falha": "Eficiencia energetica do motor",
            "Nota": "91000194175",
            "Fonte": "Notas de Manuten\u00e7\u00e3o SAP"
        },
        "2009-09-25": {
            "Descricao": "subs. valvulas descarga do 2o estagio",
            "Modo de Falha": "Falha no conjunto de valvulas",
            "Nota": "91000206439",
            "Fonte": "Notas de Manuten\u00e7\u00e3o SAP"
        },
        "2009-12-11": {
            "Descricao": "c302 a  falha na valvula do reciclo",
            "Modo de Falha": "Falha no conjunto de valvulas",
            "Nota": "91000217970",
            "Fonte": "Notas de Manuten\u00e7\u00e3o SAP"
        },
        "2010-03-11": {
            "Descricao": "sc302a vazamento jw pela resistencia el.",
            "Modo de Falha": "Falha no conjunto de valvulas",
       